
## Problem Statement

Food Delivery services like Zomato and Swiggy need to show the accurate time it will take to deliver your order to keep transparency with their customers. These companies use Machine Learning algorithms to predict the food delivery time based on how much time the delivery partners took for the same distance in the past.

To predict the food delivery time in real-time, we need to calculate the distance between the food preparation point and the point of food consumption. After finding the distance between the restaurant and the delivery locations, we need to find relationships between the time taken by delivery partners to deliver the food in the past for the same distance.


In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from math import sqrt, radians, sin , cos , atan2 
import warnings
import folium

warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler , OneHotEncoder , OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split , GridSearchCV

In [2]:
import os
os.getcwd()

'd:\\Delivery-time-prediction\\experiment'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'd:\\Delivery-time-prediction'

In [5]:
df = pd.read_csv(r"experiment\finalTrain.csv")

In [6]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [7]:
df.shape

(45584, 20)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45584 non-null  object 
 1   Delivery_person_ID           45584 non-null  object 
 2   Delivery_person_Age          43730 non-null  float64
 3   Delivery_person_Ratings      43676 non-null  float64
 4   Restaurant_latitude          45584 non-null  float64
 5   Restaurant_longitude         45584 non-null  float64
 6   Delivery_location_latitude   45584 non-null  float64
 7   Delivery_location_longitude  45584 non-null  float64
 8   Order_Date                   45584 non-null  object 
 9   Time_Orderd                  43853 non-null  object 
 10  Time_Order_picked            45584 non-null  object 
 11  Weather_conditions           44968 non-null  object 
 12  Road_traffic_density         44983 non-null  object 
 13  Vehicle_conditio

In [9]:
df.isnull().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [10]:
df.describe()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_taken (min)
count,43730.000000,43676.000000,45584.000000,45584.000000,45584.000000,45584.000000,45584.000000,44591.000000,45584.000000
mean,29.566911,4.633774,17.017948,70.229684,17.465480,70.844161,1.023385,0.744635,26.293963
std,5.815064,0.334744,8.185674,22.885575,7.335562,21.120578,0.839055,0.572510,9.384298
min,15.000000,1.000000,-30.905562,-88.366217,0.010000,0.010000,0.000000,0.000000,10.000000
25%,25.000000,4.500000,12.933284,73.170000,12.988453,73.280000,0.000000,0.000000,19.000000
50%,30.000000,4.700000,18.551440,75.897963,18.633934,76.002574,1.000000,1.000000,26.000000
75%,35.000000,4.900000,22.728163,78.044095,22.785049,78.107044,2.000000,1.000000,32.000000
max,50.000000,6.000000,30.914057,88.433452,31.054057,88.563452,3.000000,3.000000,54.000000


In [11]:
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken (min)'],
      dtype='object')

In [12]:
for col in df.columns:
    print(f" no of unique values in {col} : {df[col].nunique()}")
    if df[col].nunique() < 20:
         print(f"unique value are : {df[col].unique()}")

 no of unique values in ID : 45584
 no of unique values in Delivery_person_ID : 1320
 no of unique values in Delivery_person_Age : 22
 no of unique values in Delivery_person_Ratings : 28
 no of unique values in Restaurant_latitude : 657
 no of unique values in Restaurant_longitude : 518
 no of unique values in Delivery_location_latitude : 4373
 no of unique values in Delivery_location_longitude : 4373
 no of unique values in Order_Date : 44
 no of unique values in Time_Orderd : 176
 no of unique values in Time_Order_picked : 193
 no of unique values in Weather_conditions : 6
unique value are : ['Fog' 'Stormy' 'Sandstorms' 'Windy' 'Cloudy' 'Sunny' nan]
 no of unique values in Road_traffic_density : 4
unique value are : ['Jam' 'High' 'Medium' 'Low' nan]
 no of unique values in Vehicle_condition : 4
unique value are : [2 1 0 3]
 no of unique values in Type_of_order : 4
unique value are : ['Snack' 'Meal' 'Drinks' 'Buffet']
 no of unique values in Type_of_vehicle : 4
unique value are : ['mo

## Feature Engineering creating new features with existing features


In [13]:
feature = []
dtypes = []
count = []
unique = []
missing = []
missing_oercentage = []

for column in df.columns:
    feature.append(column)
    count.append(len(df[column]))
    missing.append(df[column].isnull().sum())
    unique.append(df[column].nunique())
    missing_oercentage.append(df[column].isnull().sum()/df.shape[0]*100)
    dtypes.append(df[column].dtypes)

dataframe = pd.DataFrame({'feature' : feature,
                          'count' : count,
                          'missing': missing,
                          'unique' : unique,
                          'missing_percentage' : missing_oercentage,
                          'dtypes' : dtypes})
dataframe.set_index('feature')

,count,missing,unique,missing_percentage,dtypes
feature,,,,,
ID,45584,0,45584,0.000000,object
Delivery_person_ID,45584,0,1320,0.000000,object
Delivery_person_Age,45584,1854,22,4.067217,float64
Delivery_person_Ratings,45584,1908,28,4.185679,float64
Restaurant_latitude,45584,0,657,0.000000,float64
Restaurant_longitude,45584,0,518,0.000000,float64
Delivery_location_latitude,45584,0,4373,0.000000,float64
Delivery_location_longitude,45584,0,4373,0.000000,float64
Order_Date,45584,0,44,0.000000,object


In [14]:
df.drop('ID' , axis =1 , inplace = True)

In [15]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d-%m-%Y')

In [16]:
df['Order_Date']

0       2022-02-12
1       2022-02-13
2       2022-03-04
3       2022-02-13
4       2022-02-14
           ...    
45579   2022-03-24
45580   2022-02-16
45581   2022-03-11
45582   2022-03-07
45583   2022-03-02
Name: Order_Date, Length: 45584, dtype: datetime64[ns]

In [17]:
df['year']  =df['Order_Date'].dt.year
df['month']  =df['Order_Date'].dt.month
df['day']  =df['Order_Date'].dt.day


In [18]:
df.head()

,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,...,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),year,month,day
0,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55,22:10,...,2,Snack,motorcycle,3.0,No,Metropolitian,46,2022,2,12
1,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,2022-02-13,14:55,15:05,...,1,Meal,motorcycle,1.0,No,Metropolitian,23,2022,2,13
2,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,2022-03-04,17:30,17:40,...,1,Drinks,scooter,1.0,No,Metropolitian,21,2022,3,4
3,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,2022-02-13,09:20,09:30,...,0,Buffet,motorcycle,0.0,No,Metropolitian,20,2022,2,13
4,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,2022-02-14,19:50,20:05,...,1,Snack,scooter,1.0,No,Metropolitian,41,2022,2,14


In [20]:
df.drop(columns=['Order_Date'] , axis= 1 , inplace= True)

In [21]:
df.head()

,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Time_Orderd,Time_Order_picked,Weather_conditions,...,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),year,month,day
0,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,21:55,22:10,Fog,...,2,Snack,motorcycle,3.0,No,Metropolitian,46,2022,2,12
1,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,14:55,15:05,Stormy,...,1,Meal,motorcycle,1.0,No,Metropolitian,23,2022,2,13
2,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,17:30,17:40,Sandstorms,...,1,Drinks,scooter,1.0,No,Metropolitian,21,2022,3,4
3,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,09:20,09:30,Sandstorms,...,0,Buffet,motorcycle,0.0,No,Metropolitian,20,2022,2,13
4,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,19:50,20:05,Fog,...,1,Snack,scooter,1.0,No,Metropolitian,41,2022,2,14


In [22]:
df.dropna(subset=['Time_Orderd'] , inplace= True)

In [23]:
df['Time_Orderd'] = df['Time_Orderd'].str.replace("." , ":")

In [24]:
df['Time_Orderd'].sample(10)

20495    22:30
19007    15:15
4451     20:45
18701    22:50
11964    10:20
35601    08:40
15181    17:45
29940    19:55
33226     0:75
38116    21:30
Name: Time_Orderd, dtype: object

In [34]:
def extarct_time(x):
    try:
        t = pd.to_datetime(x,errors='coerce')
        if pd.isna(t):
            return "00:00"
        return t.time()

    except IndexError:
        return '00:00'

df['Time_Orderd'] = df['Time_Orderd'].apply(extarct_time)

In [35]:
df['Time_Orderd']

0        00:00
1        00:00
2        00:00
3        00:00
4        00:00
         ...  
45579    00:00
45580    00:00
45581    00:00
45582    00:00
45583    00:00
Name: Time_Orderd, Length: 43853, dtype: object

## target looks like Time_taken (min) (regression problem). Let’s design a clean preprocessing pipeline.

Issues in your dataset

Missing values in:

Delivery_person_Age (1854 missing)

Delivery_person_Ratings (1908 missing)

Time_Orderd (1731 missing)

Weather_conditions, Road_traffic_density, Festival, City

multiple_deliveries (993 missing)

Order_Date, Time_Orderd, Time_Order_picked are strings → need to convert to datetime/time features.

Categorical columns: Weather_conditions, Road_traffic_density, Type_of_order, Type_of_vehicle, Festival, City

IDs (ID, Delivery_person_ID) should be dropped.

Lat/long columns can be transformed into distance features.

## Preprocessing Plan

Drop identifiers (ID, Delivery_person_ID).

Handle missing values (median/mode imputation).

Convert Order_Date → extract day, month, weekday.

Convert Time_Orderd & Time_Order_picked → minutes since midnight.

Compute delivery distance from restaurant to delivery location (Haversine formula).

Encode categorical features with LabelEncoder / OneHotEncoder.

Scale numerical features.